In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

import copy

from sklearn import svm

%matplotlib inline

In [2]:
import re

In [45]:
import nltk

In [3]:
loc1 = r'C:\Users\c0w00f8.WMSC\Documents\Coursera\1. Machine Learning\machine-learning-ex6\ex6\spamTest.mat'
spamTest = loadmat(loc1)
spamTest_x = spamTest['Xtest']
spamTest_y = spamTest['ytest']

In [4]:
loc2 = r'C:\Users\c0w00f8.WMSC\Documents\Coursera\1. Machine Learning\machine-learning-ex6\ex6\spamTrain.mat'
spamTrain = loadmat(loc2)
#spamTrain
spamTrain_x = spamTrain['X']
spamTrain_y = spamTrain['y']

In [5]:
# func to read txt files
def readFile(filename):
    with open(filename, 'r') as myfile:
        content = myfile.read()
    return content

In [6]:
testRead = readFile('emailSample1.txt')
testRead

"> Anyone knows how much it costs to host a web portal ?\n>\nWell, it depends on how many visitors you're expecting.\nThis can be anywhere from less than 10 bucks a month to a couple of $100. \nYou should checkout http://www.rackspace.com/ or perhaps Amazon EC2 \nif youre running something big..\n\nTo unsubscribe yourself from this mailing list, send an email to:\ngroupname-unsubscribe@egroups.com\n\n"

In [7]:
# get vocabulary list as a matrix
def getVocabList1(filename):
    with open(filename, 'r') as myfile:
        lines = myfile.readlines()
    lines = [x.strip('\n') for x in lines]
    vlist = [x.split('\t') for x in lines]
    
    vlist = np.array(vlist)
    
    return vlist

In [134]:
testV = getVocabList1('vocab.txt')
testV[1894]

array(['1895', 'your'], dtype='<U40')

In [9]:
# get vocab list as a dictionary
def getVocabList(filename):
    vlist = dict()
    with open(filename, 'r') as myfile:
        lines = myfile.readlines()
    lines = [x.strip('\n') for x in lines]
    lines = [x.split('\t') for x in lines]
    
    for item in lines:
        vlist[item[1]] = int(item[0])
    
    return vlist

In [62]:
testV2 = getVocabList('vocab.txt')
ww = 'anyon'
#ww in testV2
#testV2[ww]
#len(testV2.keys())

1899

In [141]:
# transform txt content to words indices
def processEmail(email):
    
    # import vocabList
    vlist = getVocabList('vocab.txt')
    
    # lower-casing
    email = email.lower()
    
    # stripping html
    email = re.sub('<[^<>]+>', '', email)
    
    # normalizing numbers
    email = re.sub('[0-9]+', 'number', email)
    
    # normalizing urls
    # Look for strings starting with http:// or https://
    email = re.sub('(http|https)://[^\s]*', 'httpaddr', email)
    
    # normalizing email address
    # Look for strings with @ in the middle
    email = re.sub('[^\s]+@[^\s]+', 'emailaddr', email)
    
    # normalizing dollars
    email = re.sub('[$]+', 'dollar', email)
    
    splitter = re.compile('[\s@$/#.-:&*+=\[\]?!(){},''">_<;%\']+')
    word_indices = []
    
    for word in splitter.split(email):
        #print (word)
        word = re.sub('\W', '', word)
        ps = nltk.stem.PorterStemmer()
        word_stemmed = ps.stem(word)
        #print (word, word_stemmed)
        
        if word_stemmed in vlist:
            word_indices.append(vlist[word_stemmed])

    
    return word_indices

In [145]:
testP = processEmail(testRead)
len(testP)
#testP

53

In [48]:
www = 'shopping'
ps = nltk.stem.PorterStemmer()
ps.stem(www)

'shop'

In [199]:
# transform word indices to features
def emailFeatures(word_ind):
    # import vocabList
    vlist = getVocabList('vocab.txt')
    
    # number of features
    n = len(vlist)
    
    x = np.zeros(n)
    
    for i in word_ind:
        x[i-1] = 1
    
    x = x.T.reshape((1, n))

    return x

In [198]:
testFeature = emailFeatures(testP)
(testFeature==1).sum()

45

In [146]:
# model training func form part 1
def svmTrain(x, y, c, kernal, sig=0.01):
    y = y.flatten()
    if kernel == 'linear':
        clf = svm.SVC(kernel=kernel, C=c)
        
    if kernel == 'gaussian':
        clf = svm.SVC(kernel='rbf', C=c, gamma=1/sigma**2/2)
    
    clf.fit(x, y)
    
    return clf

In [152]:
# train the model
c = 0.1
kernel = 'linear'

model = svmTrain(spamTrain_x, spamTrain_y, c, kernel)

pred = model.predict(spamTrain_x)

In [156]:
accu_train = (pred == spamTrain_y.flatten()).sum() / spamTrain_y.shape[0]
accu_train

0.99825

In [159]:
predTest = model.predict(spamTest_x)
accu_test = (predTest == spamTest_y.flatten()).sum() / spamTest_y.shape[0]
accu_test

0.989

In [183]:
# inspect the most predictive words
w = model.coef_
ind = w.flatten().argsort()[::-1][:15]

vlist = getVocabList1('vocab.txt')
topWords = []

for i in ind:
    topWords.append(vlist[i, 1])

topWords

['our',
 'click',
 'remov',
 'guarante',
 'visit',
 'basenumb',
 'dollar',
 'will',
 'price',
 'pleas',
 'most',
 'nbsp',
 'lo',
 'ga',
 'hour']

In [200]:
# test the email
def testMain(filename, model):
    content = readFile(filename)
    word_indices = processEmail(content)
    x = emailFeatures(word_indices)
    pred = model.predict(x)
    
    return pred

In [186]:
c = 0.1
kernel = 'linear'
modelTest = svmTrain(spamTrain_x, spamTrain_y, c, kernel)

In [204]:
filename = 'spamSample2.txt'
p = testMain(filename, modelTest)
p

array([1], dtype=uint8)